# Исследование надежности заемщиков - анализ банковских данных.

**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Задача исследования** — установить зависимость семейного положения и количества детей на погашение кредита в срок. Для этого найдем ответы на четыре вопроса:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о платежеспособности клиентов  находятся в  файле `/datasets/data.csv`. О качестве данных ничего не известно. Поэтому перед понадобится обзор данных. 

Необходимо проверить данные на ошибки и оценить их влияние на исследование. Затем, на этапе предобработки найти возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы.



## Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Тип данных object (5 столбцов), int64 (5 столбцов), float64 (2 столбца).

Согласно документации к данным:

 *	`children` — количество детей в семье;
 *	`days_employed` — общий трудовой стаж в днях;
 *	`dob_years` — возраст клиента в годах;
 *	`education` — уровень образования клиента;
 *	`education_id` — идентификатор уровня образования;
 *	`family_status` — семейное положение;
 *	`family_status_id` — идентификатор семейного положения;
 *	`gender` — пол клиента;
 *	`income_type` — тип занятости;
 *	`debt` — имел ли задолженность по возврату кредитов;
 *	`total_income` — ежемесячный доход;
 *	`purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о заемщиках банка. Часть колонок описывает самого заемщика и его семью: общий трудовой стаж в днях, возраст клиента в годах, уровень образования клиента, пол клиента,  семейное положение и количество детей в семье. Остальные данные рассказывают о занятости и кредите заемщика: тип занятости, ежемесячный доход, имел ли задолженность по возврату кредитов, цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Заполнение пропусков

В первом этапе методом info() были обнаружены два столбца с одинаковым количеством пропущенных данных: **days_employed** — общий трудовой стаж в днях и **total_income** — ежемесячный доход.

Предположим, что пропущенные данные находятся в одних и тех же строчках

In [5]:
nul_str = df.loc[:, ['total_income', 'days_employed', 'income_type']]
nul_str['days_employed'] = nul_str['days_employed'].fillna(0)
nul_str['total_income'] = nul_str['total_income'].fillna(0)
nul_str = nul_str[(nul_str['days_employed'] == 0) & (nul_str['total_income'] == 0)]

len(nul_str)

2174

In [6]:
nul_str.groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

In [7]:
df.groupby('income_type')['income_type'].count()

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

Таким образом, трудовой стаж и ежемесячный доход не заполняли одновременно, предположительно, заемщик не предоставил сведения.
Пропущенные значения составляют 10 % для столбцов days_employed  и total_income. При этом в разрезе четырех типов занятости доля пропущенных значений по вышеуказанным столбцам распределена идентично.

Рассмотрим столбец **days_employed** (общий трудовой стаж в днях).

In [8]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

   В столбце **days_employed** наблюдадаются выбросы и отрицательные значения трудового стажа. Отрицательные значения выглядят как случайно поставленный знак минус (или тире), поэтому уберем знаки, так как с ними будет искажение.
   Наличие выбросов говорит о том, что распределение признака не является нормальным, а значит заполнять пропуски необходимо медианным значением, так как она  устоучива к выбросам.

In [9]:
df['days_employed'] = df['days_employed'].abs()
df = df.fillna({'days_employed': df.groupby('income_type')['days_employed'].transform('median')}) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Второй столбец с пропущенными значениями **total_income** (ежемесячный доход).

In [10]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

  В столбце **total_income** тоже фикструется наличие выбросов, следовательно заполнять пропуски будем медианным значением, так как она  устоучива к выбросам.

In [11]:
df = df.fillna({'total_income': df.groupby('income_type')['total_income'].transform('median')}) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Проверка данных на аномалии и исправления

In [12]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,67299.486032,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,139401.804684,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.549623,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1993.522017,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5347.024506,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбце **children** наблюдадаются выбросы и отрицательные значения. Найдем значения не входящие в промежуток от 0 до 5 детей.

In [13]:
df[(df['children'] < 0) | (df['children'] > 5)]['children'].unique()

array([-1, 20])

В столбце children "-1"  выглядит как случайно поставленный знак минус (или тире), а 20 как случайно приписанный "0" к двойке. Для замены используем два метода replace() и abs().

In [14]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].abs()
df[(df['children'] < 0) | (df['children'] > 5)]['children'].unique()

array([], dtype=int64)

В столбце **days_employed** от отрицательных значений избавились в предыдущем шаге, но остались выбросы.

In [15]:
days_employed_big = df[df['days_employed'] > 16425]
days_employed_big.groupby('income_type')['income_type'].count()


income_type
безработный       2
компаньон         1
пенсионер      3856
сотрудник         2
Name: income_type, dtype: int64

При выборки стажа более 45 лет, все пенсионеры и  безработные оказались в этой выборке. Таким образом скорее всего аномалия в данных связана с технической стороной.


В столбце **dob_years** минимальное значение "0", поэтому выведем строчки с возрастом менее 18 лет.

In [16]:
df[df['dob_years'] < 18]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1574.202821,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


   Все строчки оказались нулевыми, на их долю приходится 0,5% от столбца, следовательно скорее всего данные не внесли.

In [17]:
df[df['dob_years'] < 18].groupby('income_type')['income_type'].count()

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: income_type, dtype: int64

   В разрезе типов занятости нулевые значения также составляют 0,4 - 0,5% от вышеперечисленных типов. Значение не большое и существенного влияния на результаты, если брать их во внимание, не окажет.

### Изменение типов данных

In [18]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


###  Удаление дубликатов

Выведем на экран список уникальных названий столбца **education**, отсортированный в алфавитном порядке.

In [19]:
df['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Каждый уровень образования представлен тремя формами написания отличающимися друг от друга записью в разных регистрах. Для приведения к одной форме используем метод str.lower(), который приведет символы к нижнему регистру.

In [20]:
df['education'] = df['education'].str.lower()
df['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

   Найдем сумму явных дубликатов методом duplicated().

In [21]:
df.duplicated().sum()

71

   Удалим дубликаты и восстановим индексы методом reset_index() c параметром drop равным True. 

In [22]:
df = df.drop_duplicates().reset_index(drop=True)

In [23]:
df.duplicated().sum()

0

Выведем на экран список уникальных названий столбца **family_status**, отсортированный в алфавитном порядке.

In [24]:
df['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

Дубликатов в столбце **family_status** не выявлено. 
Выведем на экран список уникальных названий столбца **gender**.

In [25]:
df['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

В данных обнаружился не известный пол. Выведем строку с некорректным значением и исправим.

In [26]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [27]:
df['gender'] = df['gender'].replace('XNA', 'M')
df['gender'].sort_values().unique()

array(['F', 'M'], dtype=object)

Выведем на экран список уникальных названий столбца **income_type**.

In [28]:
df['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

Дубликатов в столбце **income_type** не выявлено. 

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим два новых датафрейма для уровня образования и семейного положения и удалим из исходного датафрейма столбцы с названиями оставив только их идентификаторы.

In [29]:
df_edu = df[['education_id', 'education']]
df_family= df[['family_status_id', 'family_status']]
df_edu = df_edu.drop_duplicates().reset_index(drop=True)
df_family = df_family.drop_duplicates().reset_index(drop=True)
display(df_edu)
display(df_family)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [30]:
df = df.drop(columns=['education', 'family_status'], axis=1)

In [31]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода

Создадим новый столбец **total_income_category**, в котором будут 5 категорий в зависимости от ежемесячного дохода кредитополучателя.

In [32]:
def total_income_group(total_income):
    if total_income <= 30000:
        return 'E'
    if 30000 < total_income <= 50000:
        return 'D'
    if 50000 < total_income <= 200000:
        return 'C'
    if 200000 < total_income <= 1000000:
        return 'B'
    return 'A'
df['total_income_category'] = df['total_income'].apply(total_income_group)
df.head()


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [33]:
df['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

### Категоризация целей кредита

Создадим новый столбец **purpose_category**, в котором будут 4 категории в зависимости от целей получения кредита. Для этого выведем на экран уникальные записи целей получения кредита и напишем функцию, которая сформирует новый столбец с использованием метода apply().

In [34]:
df['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [35]:
def purpose_group(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образов' in purpose:
        return 'получение образования'
    else:
        print(df[purpose])

df['purpose_category'] = df['purpose'].apply(purpose_group)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [36]:
df['purpose_category'].sort_values().unique()


array(['операции с автомобилем', 'операции с недвижимостью',
       'получение образования', 'проведение свадьбы'], dtype=object)

## Ответы на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

   Определим процент возврата кредита в срок в разрезе количества детей заемщика. Для этого сформируем сводную таблицу методом pivot_table().   

In [37]:
df_pivot_child = df.pivot_table(index='children', columns='debt', values='total_income', aggfunc='count')
df_pivot_child['loan_repayment_on_time'] = df_pivot_child[0] * 100 /(df_pivot_child[0] + df_pivot_child[1])
df_pivot_child.sort_values(by='loan_repayment_on_time', ascending=False)

debt,0,1,loan_repayment_on_time
children,,,
0,13028.0,1063.0,92.456178
3,303.0,27.0,91.818182
1,4410.0,445.0,90.834192
2,1926.0,202.0,90.507519
4,37.0,4.0,90.243902
5,9.0,NaN,NaN


**Вывод**:

   Наблюдается зависимость между количеством детей и возвратом кредита в срок. 
   
   Наиболее ярко зависимость  выражена от 0 до 2 детей, где процент возврата кредита в срок постепенно уменьшается в зависимости от увеличения количества детей. Данная выборка характеризуется большим объемом заемщиков, что обеспечивает ее достоверность.
   
   Если рассматривать процент возврата кредита в срок с увеличением детей с трех до пяти, то тенденция носит волнообразный характер. Увеличение процента возврата кредита в срок для трех детей по сравнению с двумя и отсутствием просроченных кредитов у заемщиков с пятью детьми, в совокупности маленьким объемом выборки (клиенты, имеющие трех и более детей составляют всего 1,8% от всех заемщиков), говорит о возможном искажении результатов.
   
   Таким образом, чем больше клиент имеет детей, тем меньше вероятность возврата кредита в срок. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

   Определим процент возврата кредита в срок в разрезе семейного положения заемщика. Для этого сформируем сводную таблицу методом pivot_table(). 

In [38]:
df_pivot_family = df.pivot_table(index=['family_status_id'], columns='debt', values='total_income', aggfunc='count')
df_pivot_family['loan_repayment_on_time'] = df_pivot_family[0] * 100 /(df_pivot_family[0] + df_pivot_family[1])
df_pivot_family.sort_values(by='loan_repayment_on_time', ascending=False)
df_pivot_family.merge(df_family, on='family_status_id', how='left')

,family_status_id,0,1,loan_repayment_on_time,family_status
0,0,11408,931,92.454818,женат / замужем
1,1,3763,388,90.652855,гражданский брак
2,2,896,63,93.430657,вдовец / вдова
3,3,1110,85,92.887029,в разводе
4,4,2536,274,90.249110,Не женат / не замужем


**Вывод**:

Наблюдается зависимость между семейным положением и возвратом кредита в срок.

Наибольший процент возврата кредита в срок у свободных заемщиков ранее состоявших в браке (на первом месте "вдовец / вдова", на втором  "в разводе") и состоящих в официальном браке на момент получения кредита ("женат / замужем").

Хуже всего возвращают свободные заемщики ("Не женат / не замужем").

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для определения зависимости уровня дохода и возрата кредита в срок в предыдущем этапе столбец **total_income** был разбит на 5 категорий в зависимости от ежемесячного дохода и записан в новый столбец **total_income_category**:

*	`Е` — 0-30000;
*	`D` — 30001-50000;
*	`C` — 50001-200000;
*	`B` — 200001-1000000;
*	`F` — 1000001 и выше.

Определим процент возврата кредита в срок в разрезе категорий. Для этого сформируем сводную таблицу методом pivot_table(). 

In [39]:
df_pivot_income = df.pivot_table(index='total_income_category', columns='debt', values='total_income', aggfunc='count')
df_pivot_income['loan_repayment_on_time'] = df_pivot_income[0] * 100 /(df_pivot_income[0] + df_pivot_income[1])
df_pivot_income.sort_values(by='loan_repayment_on_time', ascending=False)

debt,0,1,loan_repayment_on_time
total_income_category,,,
D,329,21,94.000000
B,4686,356,92.939310
A,23,2,92.000000
C,14655,1360,91.507961
E,20,2,90.909091


**Вывод**: 

Если рассматривать отдельно уровень дохода по категориям и процент возврата кредита в срок, то наблюдается зависимость между ними. Наибольший процент возврата кредита в срок при ежемесячном доходе от 30 до 50 тыс. в месяц, наименьший при доходе до 30 тыс.

Но если учесть, что объем выборки по каждой категории сильно отличается друг от друга (по категориям А и Е она совсем маленькая), то говорить о достоверности тенденций нельзя.

Таким образом, в связи с непрезентативной выборкой однозначно сделать вывод о зависимости вышеперечисленных параметров нельзя.

### Как разные цели кредита влияют на его возврат в срок?

Для определения зависимости цели кредита и возрата кредита в срок в предыдущем этапе столбец **purpose** былсгруппирован по четырем категориям в зависимости от целей и записан в новый столбец **purpose_category**:

*	`операции с автомобилем`;
*	`операции с недвижимостью`;
*	`получение образования`;
*	`проведение свадьбы`.

Определим процент возврата кредита в срок в в зависимости от целей. Для этого сформируем сводную таблицу методом pivot_table(). 

In [40]:
df_pivot_purpose = df.pivot_table(index='purpose_category', columns='debt', values='total_income', aggfunc='count')
df_pivot_purpose['loan_repayment_on_time'] = df_pivot_purpose[0] * 100 /(df_pivot_purpose[0] + df_pivot_purpose[1])
df_pivot_purpose.sort_values(by='loan_repayment_on_time', ascending=False)

debt,0,1,loan_repayment_on_time
purpose_category,,,
операции с недвижимостью,10029,782,92.766627
проведение свадьбы,2138,186,91.996558
получение образования,3643,370,90.779965
операции с автомобилем,3903,403,90.640966


**Вывод**:

Наблюдается зависимость между целями кредита и возвратом кредита в срок.

Наибольший процент возврата кредита в срок у клиентов, которые покупают недвижимость и собираются оформить официальный брак, что обусловлено тем, что клиенты с такими целями более твердо стоят на ногах, в отличии от еще получающих образование.

## Общий вывод:

Была рассмотрена зависимость факта возврата кредита в срок от четырех параметров:

*	количество детей;
*	семейное положение;
*	уровень дохода;
*	цель кредита.

Взаимосвязь установлена по трем параметрам, кроме уровня дохода в связи с непрезентативной выборкой.

**Количество детей**: чем больше клиент имеет детей, тем меньше вероятность возврата кредита в срок.

**Семейное положение**: наибольший процент возврата кредита в срок у свободных заемщиков ранее состоявших в браке (на первом месте "вдовец / вдова", на втором "в разводе") и состоящих в официальном браке на момент получения кредита ("женат / замужем").

**Цель кредита**: наибольший процент возврата кредита в срок у клиентов, которые покупают недвижимость и собираются оформить официальный брак, что обусловлено тем, что клиенты с такими целями более твердо стоят на ногах, в отличии от еще получающих образование.
